In [2]:
#import

import torch
import torch.nn as nn # nn stuff
import torch.optim as optim # optimizer
import torch.nn.functional as f # no param functions exp, tanh vs keras functional
from torch.utils.data import DataLoader # utilities in data field
import torchvision.datasets as datasets # with tfds
import torchvision.transforms as transforms

In [3]:
#set device
#conditional with .cuda.is_avaiable
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

In [4]:
#hyperparam
input_size = 28
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 1

In [5]:
#create
class CNN(nn.Module):
  def __init__(self,in_channels=1, num_classes=10):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), padding=(1,1), stride=(1,1))
    self.pool = nn.MaxPool2d(stride=(2,2), kernel_size=(2,2))
    self.conv2 = nn.Conv2d(in_channels =8, out_channels=16, kernel_size=(3,3), padding=(1,1), stride=(1,1))
    self.fc1 = nn.Linear(16*7*7, num_classes) # 7*7 28 * 28 pooled twice
  def forward(self, x):
    x = f.relu(self.conv1(x))
    x = self.pool(x)
    x = f.relu(self.conv2(x))
    x = self.pool(x)
    x = x.reshape(x.shape[0], -1)
    x = self.fc1(x)
    return x

model = CNN()
x = torch.randn(64, 1, 28, 28)
print(model(x).shape) # test out model (really just matrix mult so has shape)

torch.Size([64, 10])


In [6]:
#load data
train_dataset = datasets.MNIST(root='dataset/', train=True,
                               transform = transforms.ToTensor(),
                               download=True) # create folder , get train, download if not there
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle=True)
# batch - for batch in train_loader
#usually for i, batch in enumerate(train_loader, start=1) for index
# test on my own
test_dataset = datasets.MNIST(root='dataset/', train=False,
                              transform=transforms.ToTensor(),
                              download=True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle=True)


100%|██████████| 9912422/9912422 [00:00<00:00, 367218209.51it/s]

Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 27606128.95it/s]


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 96136831.67it/s]

Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw




100%|██████████| 4542/4542 [00:00<00:00, 13312738.48it/s]


Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



In [7]:
#init
#tensor .to() -> device
model = CNN(num_classes=num_classes).to(device)

In [8]:
#loss and opt
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate) # learning rate
# Adam on model params

In [9]:
#train
for epoch in range(num_epochs):
  for batch_ids, (data, targets) in enumerate(train_loader):
    # get data to cuda if pos
    data = data.to(device=device)
    targets = targets.to(device=device)

    #print(data.shape) #64, 1 for color, 28x28
    #set correct shape
    #data = data.reshape(data.shape[0], -1)

    scores = model(data)
    loss = criterion(scores, targets)

    optimizer.zero_grad() # rezero
    loss.backward() # get losses # tf.backward similar

    optimizer.step() # steps through grad




In [10]:
#eval#
def check_accuracy(loader, model):
  if loader.dataset.train: # check if train bool
    print("Checking accuracy on training data")
  else:
    print("Checking accuracy on test data")
  num_correct = 0
  num_samples = 0
  model.eval()
  with torch.no_grad(): # no any grads here
    for x, y in loader:
      x = x.to(device=device)
      y = y.to(device=device)
      #x = x.reshape(x.shape[0], -1)

      scores = model(x)
      _, predictions = scores.max(1) # get high prob, gives index
      num_correct += (predictions == y).sum() # where indexes are correct
      num_samples += predictions.size(0)  # for batch
  print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples) * 100:.2f}')
  model.train()
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 57760 / 60000 with accuracy 96.27
Checking accuracy on test data
Got 9659 / 10000 with accuracy 96.59


In [11]:
# CIFAR so far 100 of these set the bar.

import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from torchvision.transforms import ToTensor


In [12]:
#set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [45]:
#hyp
input_size = 32
batch_size = 32
num_epochs = 5
num_classes = 100
lr = 0.001

In [93]:
my_channels_indices = [3, 8, 16, 32]
class CifarCNN(nn.Module):
  def __init__(self, num_classes=100):
    super(CifarCNN, self).__init__()
    self.conv_layers = [nn.Conv2d(in_channels = my_channels_indices[i], out_channels=my_channels_indices[i+1], kernel_size = (3,3), stride = (1,1), padding=(1,1)) for i in range(len(my_channels_indices) -1)]
    self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
    self.output = nn.Linear(my_channels_indices[len(my_channels_indices) - 1] * 4*4, num_classes)
  def forward(self, x):
    for i in range(len(self.conv_layers)):
      x = self.conv_layers[i](x)
      x = self.pool(x)
    x = x.reshape(x.shape[0],-1)
    return self.output(x)
model = CifarCNN()
rand_input = torch.randn(32,3, 32, 32)
model(rand_input)


tensor([[-1.1737e-01,  1.4335e-01, -1.1466e-01,  ...,  2.0476e-02,
          3.4558e-02, -6.5831e-02],
        [-3.1231e-02,  1.5097e-01, -9.6956e-02,  ...,  7.0900e-02,
          6.6000e-02, -8.5661e-02],
        [-1.1928e-01,  9.6439e-02, -1.3558e-01,  ..., -4.1118e-02,
          8.9791e-02, -8.9281e-02],
        ...,
        [-3.0985e-02,  9.9687e-02, -6.9807e-02,  ..., -4.7876e-02,
          4.2326e-02, -8.0176e-03],
        [-1.0332e-01,  8.4782e-02, -9.5599e-02,  ..., -1.4293e-04,
          1.5712e-01, -1.6272e-01],
        [-6.0029e-02,  1.0859e-01, -1.3341e-01,  ..., -2.1058e-02,
          3.3074e-02, -1.2555e-01]], grad_fn=<AddmmBackward0>)

In [118]:
#load
train_dst = datasets.CIFAR100('datasets2/', train=True, download=True, transform=ToTensor())
train_loader = DataLoader(train_dst, shuffle=True, batch_size=batch_size)

test_dst = datasets.CIFAR100('datasets2/', train=False, download=True,  transform=ToTensor())
test_loader = DataLoader(test_dst, shuffle=True, batch_size=batch_size)

Files already downloaded and verified
Files already downloaded and verified


In [108]:
#init - model init -> device
model = CifarCNN(num_classes = num_classes).to(device=device)

In [109]:
#loss+opt
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.NAdam(model.parameters(), lr=lr)

In [ ]:
#train
num_epochs=5
for i in range(num_epochs):
  print(f'Epoch #{i+1}')
  for j, (data_batch_data, data_batch_targets) in enumerate(train_loader):

    x = data_batch_data.to(device=device)
    y = data_batch_targets.to(device=device)
    scores = model(x)
    loss = criterion(scores, data_batch_targets)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

Epoch #1
Epoch #2
Epoch #3
Epoch #4


In [113]:
def check_accuracy(loader,model ):
  num_correct = 0
  num_samples = 0
  if loader.dataset.train:
    print("training")
  else:
    print("testing")
  model.eval()
  for batch in loader:
    with torch.no_grad():
      x, y = batch
      x = x.to(device=device)
      y = y.to(device)

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += x.size(0)
  print(f'{float(num_correct)/float(num_samples)*100:.2f}')
  model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

training
16.57
testing
15.73
